# 0. Libraries

## 0.1. Installing

In [1]:
!pip install agno google-genai requests pypdf chromadb -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.6/44.6 kB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 5.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 996.6/996.6 kB 45.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.5/310.5 kB 19.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.8/19.8 MB 87.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.2/284.2 kB 21.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 68.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.3/103.3 kB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.5/16.5 MB 93.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.5/72.5 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.4/105.4 kB 6.7 MB/s e

## 0.2. Importing

In [2]:
import requests

import numpy as np

from agno.agent import Agent, RunResponse, RunResponseEvent
from agno.models.google import Gemini
from agno.utils.pprint import pprint_run_response
from agno.knowledge.pdf import PDFKnowledgeBase, PDFReader
from agno.vectordb.chroma import ChromaDb
from agno.embedder.google import GeminiEmbedder

# from typing import Iterator

from pydantic import BaseModel, Field

## 0.3. API Key

In [3]:
import os

os.environ["GOOGLE_API_KEY"] = "GOOGLE_API_KEY"

# 1.0. Set Up

In [4]:
url = "https://www.ccee.org.br/documents/80415/30453320/88%20-%20InfoBandeira%20Tarif%C3%A1ria_2025_08.pdf/e2092968-bfb7-5d77-9167-d0317920a943"

In [5]:
response = requests.get(url)

if response.status_code == 200:
  with open("Bandeira.pdf", "wb") as f:
    f.write(response.content)

## 1.1. Create Knowledge Base

In [6]:
chroma_db = ChromaDb(
    embedder=GeminiEmbedder(),
    collection="bandeira",
    path="tmp/chromadb",
    persistent_client=True
)

In [58]:
pdf_kb = PDFKnowledgeBase(
    path=[
        {
            "path": "Bandeira.pdf",
            "metadata": {
                "institution": "CCEE",
                "year": 2025,
            },
        }
    ],
    reader=PDFReader(chunk=True, chunk_size=1000),
    num_documents=5,
    vector_db=chroma_db
)

In [59]:
pdf_kb

PDFKnowledgeBase(reader=PDFReader(chunk=True, chunk_size=1000, separators=['\n', '\n\n', '\r', '\r\n', '\n\r', '\t', ' ', '  '], chunking_strategy=<agno.document.chunking.fixed.FixedSizeChunking object at 0x7b22646d7050>), vector_db=<agno.vectordb.chroma.chromadb.ChromaDb object at 0x7b226bd2a1b0>, num_documents=5, optimize_on=1000, chunking_strategy=None, valid_metadata_filters=None, path=[{'path': 'Bandeira.pdf', 'metadata': {'institution': 'CCEE', 'year': 2025}}], formats=['.pdf'], exclude_files=[])

In [60]:
pdf_kb.load(recreate=True)

INFO Dropping collection

INFO Creating collection

INFO Loading knowledge base

INFO Reading: Bandeira

INFO Adding metadata {'institution': 'CCEE', 'year': 2025} to document: Bandeira

...

INFO Adding metadata {'institution': 'CCEE', 'year': 2025} to document: Bandeira

INFO Adding metadata {'institution': 'CCEE', 'year': 2025} to document: Bandeira

INFO Adding metadata {'institution': 'CCEE', 'year': 2025} to document: Bandeira

INFO Adding metadata {'institution': 'CCEE', 'year': 2025} to document: Bandeira

INFO Added 6 documents to knowledge base

# 1.2. Create Agent

In [61]:
agent = Agent(
    model=Gemini(id="gemini-2.0-flash"),
    instructions="""You are an assistant for question-answering tasks.
    Use the pieces of retrieved context to answer the question.""",
    knowledge=pdf_kb,
    markdown=True
)

# 2.0. Testing

In [62]:
agent.print_response(
    "Qual é a cor da bandeira tarifária para agosto de 2025?",
    stream=True
)

Output()

INFO Found 5 documents

In [42]:
agent.print_response(
    "Qual foi o valor do PLD gatilho considerado para agosto de 2025?",
    stream=True
)

Output()

INFO Found 5 documents

In [ ]:
agent.print_response(
    "Os valores de expectativa de geração das usinas hidráulicas que balizam o cálculo do GH_band são encaminhados por qual orgão?",
    stream=True
)

Output()

INFO Found 5 documents

In [ ]:
agent.print_response(
    "Como foi feita a definição da bandeira tarifária para Agosto de 2025?",
    stream=True
)

Output()

INFO Found 5 documents

In [ ]:
agent.print_response(
    "Dado que meu PLD gatilho foi de R$ 100/MWh, qual a cor da bandeira?",
    stream=True
)

Output()

# 3.0. Implementing a Custom Tool

In [ ]:
def classify_flag(pld_trigger: float) -> str:
  """
  Dado um valor de PLD gatilho, retorna a bandeira
  """

  if pld_trigger < 58.6 or pld_trigger > 751.73:
    return "Erro na classificação, valor fora do range possível."

  if pld_trigger >= 58.6 and pld_trigger <= 74.27:
    return "Verde"

  elif pld_trigger > 74.27 and pld_trigger <= 186.46:
    return "Amarela"

  elif pld_trigger > 186.46 and pld_trigger <= 256.9:
    return "Vermelha 1"

  else:
    return "Vermelha 2"

In [ ]:
agent = Agent(
    model=Gemini(id="gemini-2.0-flash"),
    tools=[classify_flag],
    knowledge=pdf_kb,
    markdown=True
)

In [ ]:
agent.print_response(
    "Dado que o PLD gatilho foi de R$ 100/MWh, qual a cor da bandeira?",
    stream=True
)

Output()